In [1]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

# Parse the JSON and preview it
print(type(data), len(data))

def json_print(data):
    print(json.dumps(data, indent=2))

json_print(data[0])

<class 'list'> 10
{
  "Category": "SCIENCE",
  "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
  "Answer": "Liver"
}


In [105]:
import weaviate

client = weaviate.connect_to_local()

print(client.is_ready())

# client.close()

True


In [106]:
client.get_meta()

{'grpcMaxMessageSize': 104858000,
 'hostname': 'http://[::]:8080',
 'modules': {'generative-anthropic': {'documentationHref': 'https://docs.anthropic.com/en/api/getting-started',
   'name': 'Generative Search - Anthropic'},
  'generative-anyscale': {'documentationHref': 'https://docs.anyscale.com/endpoints/overview',
   'name': 'Generative Search - Anyscale'},
  'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html',
   'name': 'Generative Search - AWS'},
  'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/chat',
   'name': 'Generative Search - Cohere'},
  'generative-databricks': {'documentationHref': 'https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html#completion-task',
   'name': 'Generative Search - Databricks'},
  'generative-friendliai': {'documentationHref': 'https://docs.friendli.ai/openapi/create-chat-completions',
   'name': 'Generative Search - FriendliAI'},
 

In [113]:
from weaviate.classes.config import Configure
  
if client.collections.exists("DemoCollection"):
    client.collections.delete("DemoCollection")
  
client.collections.create(
    "DemoCollection",
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(     # Configure the Ollama embedding integration
        api_endpoint="http://host.docker.internal:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="nomic-embed-text",                               # The model to use
    ),
    generative_config=Configure.Generative.ollama(              # Configure the Ollama generative integration
        api_endpoint="http://host.docker.internal:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="llama3.2",                                       # The model to use
    )
)

In [114]:
collection = client.collections.get("DemoCollection")

In [131]:
from weaviate.util import generate_uuid5 

with collection.batch.dynamic() as batch:
    for i, d in enumerate(data):
        
        properties = {
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        }
        
        object_uuid = generate_uuid5(properties)
        print(f"importing question: {i+1} -> UUID {object_uuid}")
        print(properties)
        print()
        
        batch.add_object(
            properties = properties,
            uuid = object_uuid
        )
        
        if batch.number_errors > 10:
            print("Batch import stopped due to excessive errors.")
            break

importing question: 1 -> UUID 2e3d7a13-f97d-56a7-8237-e3b2ffdd8ce3
{'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}

importing question: 2 -> UUID 75e0f50c-bddd-5d88-ae87-bb0a3133947a
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}

importing question: 3 -> UUID 01ae010e-f033-5ba1-9541-7efa117c8aef
{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}

importing question: 4 -> UUID 2ba855d5-89ee-5eb0-995c-362a2b6b1090
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}

importing question: 5 -> UUID 5cf0152e-3c9d-5279-8598-7ab034e42f58
{'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'categor

In [132]:
failed_objects = collection.batch.failed_objects
if failed_objects:
    print(f"Number of failed imports: {len(failed_objects)}")
    print(f"First failed object: {failed_objects[0]}")

In [133]:
for item in collection.iterator():
    print(item.uuid, item.properties)

01ae010e-f033-5ba1-9541-7efa117c8aef {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}
2ba855d5-89ee-5eb0-995c-362a2b6b1090 {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
2e3d7a13-f97d-56a7-8237-e3b2ffdd8ce3 {'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
40e17988-c987-5794-87ca-134ae8395fd9 {'answer': 'the atmosphere', 'question': 'Changes in the tropospheric layer of this are what gives us weather', 'category': 'SCIENCE'}
5cf0152e-3c9d-5279-8598-7ab034e42f58 {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'}
75e0f50c-bddd-5d88-ae87-bb0a3133947a {'answer': 'Elephant', 'question': "It's the only living mammal in the order Prob

In [136]:
response = collection.query.fetch_objects()

for o in response.objects:
    print(o.properties)

{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
{'answer': 'the atmosphere', 'question': 'Changes in the tropospheric layer of this are what gives us weather', 'category': 'SCIENCE'}
{'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'}
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
{'answer': 'species', 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'}
{'answer': 'Sou

In [134]:
collection.query.fetch_objects(
    include_vector=True,
    limit = 2
)

QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('01ae010e-f033-5ba1-9541-7efa117c8aef'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}, references=None, vector={'default': [-0.02467971295118332, 0.01199500635266304, -0.1837640404701233, -0.0886145532131195, 0.07046795636415482, 0.05945352092385292, -0.023709459230303764, 0.00413313927128911, -0.04040379077196121, 0.017029475420713425, 0.011455995962023735, 0.048080600798130035, 0.06120479106903076, 0.04060178995132446, 0.04720541462302208, -0.07246647030115128, -0.03242824226617813, -0.06466324627399445, 0.06444982439279556, 0.07792709022760391, -0.04310234636068344, 0.028867101296782494, 0.021477654576301575, 0.06514472514390945, 0.09722220152616501, 0.00218

In [135]:
response = collection.aggregate.over_all(total_count=True)

print(response.total_count)

10


In [138]:
response = collection.query.fetch_objects(
    limit=1,
    return_properties=["question", "answer"]
)

for o in response.objects:
    print(o.properties)

{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature'}


In [139]:
from weaviate.classes.query import MetadataQuery

response = collection.query.near_text(
    query="animals in movies",
    limit=2,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.distance)

{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
0.42339658737182617
{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
0.42683273553848267


In [140]:
response

QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('75e0f50c-bddd-5d88-ae87-bb0a3133947a'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.42339658737182617, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}, references=None, vector={}, collection='DemoCollection'), Object(uuid=_WeaviateUUIDInt('2ba855d5-89ee-5eb0-995c-362a2b6b1090'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.42683273553848267, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}, references=None, vector={}, collection='DemoCollection')])

In [141]:
response = collection.query.hybrid(
    query="food",
    alpha=0.5,
    return_metadata=MetadataQuery(score=True, explain_score=True),
    limit=3,
)

for o in response.objects:
    print(o.properties)
    print(o.metadata.score, o.metadata.explain_score)

{'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
0.5 
Hybrid (Result Set vector,hybridVector) Document 2e3d7a13-f97d-56a7-8237-e3b2ffdd8ce3: original score 0.48939443, normalized score: 0.5
{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}
0.3862789571285248 
Hybrid (Result Set vector,hybridVector) Document 01ae010e-f033-5ba1-9541-7efa117c8aef: original score 0.47480273, normalized score: 0.38627896
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
0.19257158041000366 
Hybrid (Result Set vector,hybridVector) Document 75e0f50c-bddd-5d88-ae87-bb0a3133947a: original score 0.4499479, normalized score: 0.19257158


In [143]:
# Set weight to properties values
response = collection.query.hybrid(
    query="animal",
    query_properties=["question^2", "answer"],
    alpha=0.25,
    limit=3,
)

for o in response.objects:
    print(o.properties)

{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}
{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}
